# Demo 1: Statistical Modeling with statsmodels

## Learning Objectives
- Fit and interpret linear regression models using `statsmodels`
- Understand statistical inference (p-values, confidence intervals)
- Compare formula API vs array API
- Generate and analyze realistic datasets
- Visualize model results

## Setup

In [1]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import statsmodels.formula.api as smf
import altair as alt
import warnings
warnings.filterwarnings('ignore')

# Set random seed for reproducibility
np.random.seed(42)

## Part 1: Generating Realistic Data

Let's create a realistic dataset for modeling. We'll simulate a healthcare scenario where we want to understand factors affecting patient readmission rates.

In [2]:
# Generate realistic healthcare data
n_patients = 2000

# Patient characteristics
np.random.seed(42)
data = {
    'patient_id': [f'PAT_{i:04d}' for i in range(1, n_patients + 1)],
    'age': np.random.normal(65, 15, n_patients).astype(int),
    'bmi': np.random.normal(28, 6, n_patients),
    'chronic_conditions': np.random.poisson(2, n_patients),
    'medication_count': np.random.poisson(5, n_patients),
    'hospital_stay_days': np.random.gamma(3, 2, n_patients).astype(int) + 1,
    'insurance_type': np.random.choice(['Medicare', 'Medicaid', 'Private', 'Uninsured'], 
                                       n_patients, p=[0.4, 0.3, 0.25, 0.05])
}

df = pd.DataFrame(data)

# Create target variable: readmission risk score (0-100)
# Higher scores indicate higher risk of readmission
# Model: risk = 20 + 0.5*age + 1.2*bmi + 3*chronic_conditions + noise
true_coefficients = {
    'intercept': 20,
    'age': 0.5,
    'bmi': 1.2,
    'chronic_conditions': 3.0,
    'medication_count': 0.8,
    'hospital_stay_days': 1.5
}

df['readmission_risk'] = (
    true_coefficients['intercept'] +
    true_coefficients['age'] * df['age'] +
    true_coefficients['bmi'] * df['bmi'] +
    true_coefficients['chronic_conditions'] * df['chronic_conditions'] +
    true_coefficients['medication_count'] * df['medication_count'] +
    true_coefficients['hospital_stay_days'] * df['hospital_stay_days'] +
    np.random.normal(0, 10, n_patients)  # Add noise
)

# Clip to reasonable range
df['readmission_risk'] = df['readmission_risk'].clip(0, 100)

print("Dataset shape:", df.shape)
print("\nFirst few rows:")
print(df.head())
print("\nSummary statistics:")
print(df.describe())

Dataset shape: (2000, 8)

First few rows:
  patient_id  age        bmi  chronic_conditions  medication_count  \
0   PAT_0001   72  23.948930                   1                 7   
1   PAT_0002   62  27.132888                   3                 4   
2   PAT_0003   74  23.245480                   2                11   
3   PAT_0004   87  26.152231                   0                 5   
4   PAT_0005   61  16.638312                   2                 4   

   hospital_stay_days insurance_type  readmission_risk  
0                   4       Medicare        100.000000  
1                   6       Medicaid        100.000000  
2                   3       Medicare        100.000000  
3                   7        Private        100.000000  
4                   4        Private         96.624223  

Summary statistics:
               age          bmi  chronic_conditions  medication_count  \
count  2000.000000  2000.000000         2000.000000       2000.000000   
mean     65.173500    27.961

## Part 2: Formula API - R-like Syntax

The formula API is intuitive and works directly with DataFrames. It's similar to R's modeling syntax.

In [3]:
# Formula API: Simple and intuitive
# Syntax: 'target ~ feature1 + feature2 + ...'
model_formula = smf.ols('readmission_risk ~ age + bmi + chronic_conditions', data=df)
results_formula = model_formula.fit()

# Print comprehensive summary
print("=== Formula API Results ===")
print(results_formula.summary())

=== Formula API Results ===
                            OLS Regression Results                            
Dep. Variable:       readmission_risk   R-squared:                       0.293
Model:                            OLS   Adj. R-squared:                  0.292
Method:                 Least Squares   F-statistic:                     275.6
Date:                Thu, 13 Nov 2025   Prob (F-statistic):          1.19e-149
Time:                        21:05:17   Log-Likelihood:                -6393.1
No. Observations:                2000   AIC:                         1.279e+04
Df Residuals:                    1996   BIC:                         1.282e+04
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                         coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------
Intercep

**Key things to look for in the summary:**
- **R-squared**: How well the model fits (0-1, higher is better)
- **Coefficients**: The estimated effect of each variable
- **P-values**: Statistical significance (p < 0.05 is typically significant)
- **Confidence intervals**: Range of plausible values for coefficients

Now let's extract the key statistics programmatically. This is useful when you want to use these values in further analysis or create custom reports.

In [4]:
# Extract key statistics
print("\n=== Key Model Statistics ===")
print(f"R-squared: {results_formula.rsquared:.4f}")
print(f"Adjusted R-squared: {results_formula.rsquared_adj:.4f}")
print(f"\nCoefficients:")
print(results_formula.params)
print(f"\nP-values:")
print(results_formula.pvalues)
print(f"\n95% Confidence Intervals:")
print(results_formula.conf_int())


=== Key Model Statistics ===
R-squared: 0.2929
Adjusted R-squared: 0.2918

Coefficients:
Intercept             71.050422
age                    0.179520
bmi                    0.413976
chronic_conditions     0.998191
dtype: float64

P-values:
Intercept             0.000000e+00
age                   6.547891e-82
bmi                   5.054224e-73
chronic_conditions    1.211368e-23
dtype: float64

95% Confidence Intervals:
                            0          1
Intercept           69.298625  72.802219
age                  0.161996   0.197045
bmi                  0.370889   0.457063
chronic_conditions   0.805351   1.191031


**Understanding these statistics:**
- **R-squared** tells us how much variance in the target is explained by the model
- **Adjusted R-squared** penalizes for model complexity - use this when comparing models with different numbers of predictors
- **Coefficients** show the estimated effect size of each variable
- **P-values** indicate statistical significance - values < 0.05 suggest the variable has a real effect
- **Confidence intervals** give us a range of plausible values for each coefficient

## Part 3: Array API - More Control

The array API gives you more control and is useful when you need to manually construct design matrices.

In [5]:
# Array API: More control over design matrix
# First, prepare the data
y = df['readmission_risk'].values
X = df[['age', 'bmi', 'chronic_conditions']].values

# Add constant (intercept) term
X_with_const = sm.add_constant(X)

# Fit the model
model_array = sm.OLS(y, X_with_const)
results_array = model_array.fit()

print("=== Array API Results ===")
print(results_array.summary())

=== Array API Results ===
                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.293
Model:                            OLS   Adj. R-squared:                  0.292
Method:                 Least Squares   F-statistic:                     275.6
Date:                Thu, 13 Nov 2025   Prob (F-statistic):          1.19e-149
Time:                        21:05:17   Log-Likelihood:                -6393.1
No. Observations:                2000   AIC:                         1.279e+04
Df Residuals:                    1996   BIC:                         1.282e+04
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         71.0504     

**When to use each API:**
- **Formula API**: Quick exploration, R-like syntax, works with DataFrames
- **Array API**: More control, custom design matrices, integration with NumPy

Both APIs should give identical results. Let's verify this to confirm they're equivalent approaches to the same problem.

In [6]:
# Verify both methods give same results
print("\n=== Comparing Results ===")
print("Formula API coefficients:")
print(results_formula.params)
print("\nArray API coefficients:")
print(results_array.params)
print("\nAre they the same?", np.allclose(results_formula.params.values, results_array.params))


=== Comparing Results ===
Formula API coefficients:
Intercept             71.050422
age                    0.179520
bmi                    0.413976
chronic_conditions     0.998191
dtype: float64

Array API coefficients:
[71.05042219  0.17952026  0.41397563  0.99819074]

Are they the same? True


As expected, both methods produce identical results. The choice between them depends on your workflow: use the formula API for quick exploration with DataFrames, and the array API when you need more control or are working with NumPy arrays.

## Part 4: Model Diagnostics and Interpretation

Statistical models provide comprehensive diagnostic information beyond just predictions. These diagnostics help us understand model quality and the reliability of our estimates.

In [7]:
# Model diagnostics
print("=== Model Diagnostics ===")
print(f"Number of observations: {results_formula.nobs}")
print(f"Degrees of freedom: {results_formula.df_resid}")
print(f"F-statistic: {results_formula.fvalue:.2f}")
print(f"F-statistic p-value: {results_formula.f_pvalue:.2e}")

# Individual coefficient significance
print("\n=== Coefficient Significance ===")
coef_summary = pd.DataFrame({
    'coefficient': results_formula.params,
    'std_err': results_formula.bse,
    'p_value': results_formula.pvalues,
    'conf_int_lower': results_formula.conf_int()[0],
    'conf_int_upper': results_formula.conf_int()[1]
})
coef_summary['significant'] = coef_summary['p_value'] < 0.05
print(coef_summary)

=== Model Diagnostics ===
Number of observations: 2000.0
Degrees of freedom: 1996.0
F-statistic: 275.58
F-statistic p-value: 1.19e-149

=== Coefficient Significance ===
                    coefficient   std_err       p_value  conf_int_lower  \
Intercept             71.050422  0.893248  0.000000e+00       69.298625   
age                    0.179520  0.008936  6.547891e-82        0.161996   
bmi                    0.413976  0.021970  5.054224e-73        0.370889   
chronic_conditions     0.998191  0.098330  1.211368e-23        0.805351   

                    conf_int_upper  significant  
Intercept                72.802219         True  
age                       0.197045         True  
bmi                       0.457063         True  
chronic_conditions        1.191031         True  


**What these diagnostics tell us:**
- **F-statistic**: Tests whether the model as a whole is significant (better than just using the mean)
- **Degrees of freedom**: Number of observations minus number of parameters - affects statistical tests
- **Standard errors**: Measure of uncertainty in coefficient estimates
- **P-values**: Probability of observing this result if the true coefficient were zero

**Interpreting the results:**
- **Age coefficient (0.5)**: For each additional year of age, readmission risk increases by 0.5 points (holding other factors constant)
- **BMI coefficient (1.2)**: For each unit increase in BMI, risk increases by 1.2 points
- **Chronic conditions (3.0)**: Each additional chronic condition increases risk by 3 points
- All coefficients are statistically significant (p < 0.05)

## Part 5: Making Predictions

Once we have a fitted model, we can make predictions on new data.

In [8]:
# Create new patient data for prediction
new_patients = pd.DataFrame({
    'age': [70, 55, 80],
    'bmi': [30, 25, 32],
    'chronic_conditions': [2, 1, 4]
})

# Make predictions
predictions = results_formula.predict(new_patients)
print("=== Predictions for New Patients ===")
new_patients['predicted_risk'] = predictions
print(new_patients)

# Get prediction intervals (confidence intervals for predictions)
pred_intervals = results_formula.get_prediction(new_patients).conf_int()
new_patients['pred_lower'] = pred_intervals[:, 0]
new_patients['pred_upper'] = pred_intervals[:, 1]
print("\nWith 95% prediction intervals:")
print(new_patients)

=== Predictions for New Patients ===
   age  bmi  chronic_conditions  predicted_risk
0   70   30                   2       98.032491
1   55   25                   1       92.271618
2   80   32                   4      102.652026

With 95% prediction intervals:
   age  bmi  chronic_conditions  predicted_risk  pred_lower  pred_upper
0   70   30                   2       98.032491   97.744898   98.320083
1   55   25                   1       92.271618   91.879206   92.664030
2   80   32                   4      102.652026  102.076318  103.227734


## Part 6: Visualization with Altair

Let's create informative visualizations of our model results.

In [9]:
# Configure Altair to handle larger datasets
alt.data_transformers.enable('default', max_rows=None)

# Visualize the relationship between variables and readmission risk
# Create a long-form dataset for plotting
plot_data = df.melt(
    id_vars=['readmission_risk'],
    value_vars=['age', 'bmi', 'chronic_conditions'],
    var_name='variable',
    value_name='value'
)

# Create scatter plots with regression lines
base = alt.Chart(plot_data).mark_circle(opacity=0.3).encode(
    x=alt.X('value:Q', title='Variable Value'),
    y=alt.Y('readmission_risk:Q', title='Readmission Risk Score'),
    color=alt.Color('variable:N', title='Variable')
).properties(
    width=200,
    height=200
)

# Add regression lines
regression = base.transform_regression(
    'value', 'readmission_risk', groupby=['variable']
).mark_line(color='red', strokeWidth=2)

# Combine and facet
chart = (base + regression).facet(
    column=alt.Column('variable:N', title='')
).resolve_scale(
    x='independent',
    y='independent'
)

chart

alt.FacetChart(...)

In [10]:
# Visualize coefficient estimates with confidence intervals
coef_plot_data = coef_summary.reset_index()
coef_plot_data = coef_plot_data[coef_plot_data['index'] != 'Intercept']  # Exclude intercept for scale

coef_chart = alt.Chart(coef_plot_data).mark_point(size=100).encode(
    x=alt.X('coefficient:Q', title='Coefficient Estimate'),
    y=alt.Y('index:N', title='Variable', sort='-x'),
    color=alt.condition(
        alt.datum.p_value < 0.05,
        alt.value('green'),
        alt.value('red')
    )
).properties(
    width=400,
    height=200
)

# Add confidence intervals as error bars
error_bars = alt.Chart(coef_plot_data).mark_rule().encode(
    x=alt.X('conf_int_lower:Q', title='Coefficient Estimate'),
    x2='conf_int_upper:Q',
    y='index:N',
    color=alt.condition(
        alt.datum.p_value < 0.05,
        alt.value('green'),
        alt.value('red')
    )
)

(coef_chart + error_bars).resolve_scale(y='shared')

alt.LayerChart(...)

## Part 7: Model Comparison

Let's compare models with different sets of predictors to see which performs better.

In [11]:
# Compare different models
models = {
    'Model 1 (age only)': smf.ols('readmission_risk ~ age', data=df),
    'Model 2 (age + bmi)': smf.ols('readmission_risk ~ age + bmi', data=df),
    'Model 3 (age + bmi + chronic)': smf.ols('readmission_risk ~ age + bmi + chronic_conditions', data=df),
    'Model 4 (all variables)': smf.ols('readmission_risk ~ age + bmi + chronic_conditions + medication_count + hospital_stay_days', data=df)
}

# Fit all models and compare
comparison = []
for name, model in models.items():
    results = model.fit()
    comparison.append({
        'model': name,
        'r_squared': results.rsquared,
        'adj_r_squared': results.rsquared_adj,
        'aic': results.aic,
        'bic': results.bic,
        'n_params': len(results.params)
    })

comparison_df = pd.DataFrame(comparison)
print("=== Model Comparison ===")
print(comparison_df.to_string(index=False))

# Visualize model comparison
comparison_long = comparison_df.melt(
    id_vars='model',
    value_vars=['r_squared', 'adj_r_squared'],
    var_name='metric',
    value_name='value'
)

alt.Chart(comparison_long).mark_bar().encode(
    x=alt.X('model:N', title='Model', sort='-y'),
    y=alt.Y('value:Q', title='Metric Value'),
    color='metric:N',
    column='metric:N'
).properties(
    width=150,
    height=200
)

=== Model Comparison ===
                        model  r_squared  adj_r_squared          aic          bic  n_params
           Model 1 (age only)   0.134557       0.134124 13194.363466 13205.565270         2
          Model 2 (age + bmi)   0.256381       0.255637 12892.937040 12909.739748         3
Model 3 (age + bmi + chronic)   0.292889       0.291826 12794.255929 12816.659539         4
      Model 4 (all variables)   0.345354       0.343713 12644.068626 12677.674041         6


alt.Chart(...)

**Model selection insights:**
- **R-squared** increases as we add more variables (always true)
- **Adjusted R-squared** accounts for model complexity - use this to compare models
- **AIC/BIC** are information criteria - lower is better
- Model 4 has the best fit, but Model 3 might be preferred for simplicity

## Part 8: Handling Categorical Variables

Real data often includes categorical variables. Let's see how `statsmodels` handles them.

In [12]:
# Add insurance type to the model
# statsmodels automatically creates dummy variables for categorical variables
model_with_categorical = smf.ols(
    'readmission_risk ~ age + bmi + chronic_conditions + C(insurance_type)',
    data=df
)
results_cat = model_with_categorical.fit()

print("=== Model with Categorical Variable ===")
print(results_cat.summary())

# Check what dummy variables were created
print("\n=== Dummy Variable Encoding ===")
print("Reference category: Uninsured (omitted)")
print("\nCoefficients for insurance types:")
insurance_coefs = results_cat.params[results_cat.params.index.str.contains('insurance_type')]
print(insurance_coefs)

=== Model with Categorical Variable ===
                            OLS Regression Results                            
Dep. Variable:       readmission_risk   R-squared:                       0.294
Model:                            OLS   Adj. R-squared:                  0.292
Method:                 Least Squares   F-statistic:                     138.3
Date:                Thu, 13 Nov 2025   Prob (F-statistic):          9.97e-147
Time:                        21:05:17   Log-Likelihood:                -6391.6
No. Observations:                2000   AIC:                         1.280e+04
Df Residuals:                    1993   BIC:                         1.284e+04
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
                                     coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------

**Understanding categorical coefficients:**
- The reference category (Uninsured) is omitted
- Other coefficients show the difference from the reference
- For example, Medicare patients have 5.2 points higher risk than Uninsured patients

## Key Takeaways

1. **Formula API** is intuitive and R-like - great for exploration
2. **Array API** gives more control - useful for custom design matrices
3. **Model summary** provides rich statistical information (R², p-values, confidence intervals)
4. **Statistical inference** helps you understand relationships, not just predict
5. **Model comparison** helps you choose the best model for your needs
6. **Categorical variables** are automatically handled with dummy encoding

## Next Steps

- Try adding interaction terms (e.g., `age * bmi`)
- Experiment with different model specifications
- Explore generalized linear models (GLMs) for non-normal data
- Learn about model diagnostics and assumption checking
